In [1]:
import pypsa
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
n=pypsa.Network()

#Snapshots
n.set_snapshots(value=pd.date_range(freq="h", start="2013-01-01", end="2014-01-01", inclusive='left'))
n.snapshot_weightings[:] *= 8760.0 / n.snapshot_weightings.sum()
Nyears = n.snapshot_weightings.objective.sum() / 8760.0

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:374: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

c:\Users\denis\miniconda3\envs\pypsa-earth\lib\site-packages

In [2]:
buses = 200

for i in range(buses):
    n.add("Bus", "My bus {}".format(i), v_nom=20.0)

n.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
My bus 0,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 1,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 2,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 3,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 4,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
...,...,...,...,...,...,...,...,...,...,...,...
My bus 195,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 196,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
My bus 197,20.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,


In [3]:
# Load yaml files
import yaml

with open(r'config.yaml') as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
 
        print(config)

{'version': '0.0.2', 'tutorial': True, 'logging': {'level': 'INFO', 'format': '%(levelname)s:%(name)s:%(message)s'}, 'scenario': {'simpl': [''], 'll': ['copt'], 'clusters': [6], 'opts': ['Co2L-4H']}, 'countries': ['NG', 'BJ'], 'summary_dir': 'results', 'snapshots': {'start': '2013-03-1', 'end': '2013-03-7', 'inclusive': 'left'}, 'enable': {'retrieve_databundle': True, 'download_osm_data': False, 'build_cutout': True, 'build_natura_raster': True}, 'crs': {'geo_crs': 'EPSG:4326', 'distance_crs': 'EPSG:3857', 'area_crs': 'ESRI:54009'}, 'retrieve_databundle': {'show_progress': True}, 'augmented_line_connection': {'add_to_snakefile': False, 'connectivity_upgrade': 2, 'new_line_type': ['HVAC'], 'min_expansion': 1, 'min_DC_length': 600}, 'cluster_options': {'alternative_clustering': False, 'distribute_cluster': ['load'], 'out_logging': True, 'aggregation_strategies': {'generators': {'p_nom': 'sum', 'p_nom_max': 'sum', 'p_nom_min': 'sum', 'p_min_pu': 'mean', 'marginal_cost': 'mean', 'committab

In [4]:
costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

In [5]:
# technologies_path={
#     'onwind': "renewable_profiles/profile_onwind.nc",
#     'solar': "renewable_profiles/profile_solar.nc"
# }

In [6]:
tech_costs=pd.read_csv(r'C:\Users\denis\OneDrive\Desktop\Mini grids\pypsa-distribution\costs.csv')

idx = pd.IndexSlice

def calculate_annuity(n, r):
    """
    Calculate the annuity factor for an asset with lifetime n years and
    discount rate of r, e.g. annuity(20, 0.05) * 20 = 1.6
    """
    if isinstance(r, pd.Series):
        return pd.Series(1 / n, index=r.index).where(
            r == 0, r / (1.0 - 1.0 / (1.0 + r) ** n)
        )
    elif r > 0:
        return r / (1.0 - 1.0 / (1.0 + r) ** n)
    else:
        return 1 / n

In [7]:
tech_costs='C://Users//denis//OneDrive//Desktop//Mini grids//pypsa-distribution//costs.csv'

In [8]:
def load_costs(tech_costs, config, elec_config, Nyears=1):
    """
    set all asset costs and other parameters
    """
    costs = pd.read_csv(tech_costs, index_col=list(range(3))).sort_index()

    # correct units to MW and EUR
    costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
    costs.loc[costs.unit.str.contains("USD"), "value"] *= config["USD2013_to_EUR2013"]

    costs = (
        costs.loc[idx[:, config["year"], :], "value"]
        .unstack(level=2)
        .groupby("technology")
        .sum(min_count=1)
    )

    costs = costs.fillna(
        {
            "CO2 intensity": 0,
            "FOM": 0,
            "VOM": 0,
            "discount rate": config["discountrate"],
            "efficiency": 1,
            "fuel": 0,
            "investment": 0,
            "lifetime": 25,
        }
    )

    costs["capital_cost"] = (
        (
            calculate_annuity(costs["lifetime"], costs["discount rate"])
            + costs["FOM"] / 100.0
        )
        * costs["investment"]
        * Nyears
    )

    costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
    costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]

    costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

    costs = costs.rename(columns={"CO2 intensity": "co2_emissions"})

    costs.at["OCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]
    costs.at["CCGT", "co2_emissions"] = costs.at["gas", "co2_emissions"]

    costs.at["solar", "capital_cost"] = 0.5 * (
        costs.at["solar-rooftop", "capital_cost"]
        + costs.at["solar-utility", "capital_cost"]
    )

    def costs_for_storage(store, link1, link2=None, max_hours=1.0):
        capital_cost = link1["capital_cost"] + max_hours * store["capital_cost"]
        if link2 is not None:
            capital_cost += link2["capital_cost"]
        return pd.Series(
            dict(capital_cost=capital_cost, marginal_cost=0.0, co2_emissions=0.0)
        )

    max_hours = elec_config["max_hours"]
    costs.loc["battery"] = costs_for_storage(
        costs.loc["battery storage"],
        costs.loc["battery inverter"],
        max_hours=max_hours["battery"],
    )
    costs.loc["H2"] = costs_for_storage(
        costs.loc["hydrogen storage"],
        costs.loc["fuel cell"],
        costs.loc["electrolysis"],
        max_hours=max_hours["H2"],
    )

    for attr in ("marginal_cost", "capital_cost"):
        overwrites = config.get(attr)
        if overwrites is not None:
            overwrites = pd.Series(overwrites)
            costs.loc[overwrites.index, attr] = overwrites

    return costs

In [9]:
costs = load_costs(
    tech_costs,
    config["costs"],
    config["electricity"],
    Nyears,
    )

In [10]:

def attach_wind_and_solar(n, costs, technologies, extendable_carriers):
    
    for tech in technologies:
        
        # with xr.open_dataset(technologies_path[tech]) as ds:
        with xr.open_dataset(f"renewable_profiles/profile_{tech}.nc") as ds:

            if ds.indexes["bus"].empty:
                continue   

            suptech = tech.split("-", 2)[0]


            n.madd(
            "Generator",
            ds.indexes["bus"],
            " " + tech,
            bus=ds.indexes["bus"],
            carrier=tech,
            p_nom_extendable=tech in extendable_carriers["Generator"],
            p_nom_max=ds["p_nom_max"].to_pandas(),
            weight=ds["weight"].to_pandas(),
            marginal_cost=costs.at[suptech, "marginal_cost"],
            capital_cost=costs.at[tech, "capital_cost"],
            efficiency=costs.at[suptech, "efficiency"],
            p_max_pu=ds["profile"].transpose("time", "bus").to_pandas(),
            )


In [11]:
attach_wind_and_solar(
    n,
    costs,
    config["renewable"],
    config["electricity"]["extendable_carriers"],
    )

Index(['0 onwind', '1 onwind', '3 onwind', '4 onwind', '6 onwind', '7 onwind',
       '8 onwind', '9 onwind', '11 onwind', '12 onwind',
       ...
       '247 onwind', '248 onwind', '249 onwind', '250 onwind', '251 onwind',
       '252 onwind', '253 onwind', '254 onwind', '255 onwind', '256 onwind'],
      dtype='object', name='bus', length=222)
               '2013-01-01 02:00:00', '2013-01-01 03:00:00',
               '2013-01-01 04:00:00', '2013-01-01 05:00:00',
               '2013-01-01 06:00:00', '2013-01-01 07:00:00',
               '2013-01-01 08:00:00', '2013-01-01 09:00:00',
               ...
               '2013-12-31 14:00:00', '2013-12-31 15:00:00',
               '2013-12-31 16:00:00', '2013-12-31 17:00:00',
               '2013-12-31 18:00:00', '2013-12-31 19:00:00',
               '2013-12-31 20:00:00', '2013-12-31 21:00:00',
               '2013-12-31 22:00:00', '2013-12-31 23:00:00'],
              dtype='datetime64[ns]', name='snapshot', length=8616, freq=None) are 

In [12]:
n.generators

,bus,carrier,p_nom_extendable,p_nom_max,weight,marginal_cost,capital_cost,efficiency,control,type,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
0 onwind,0,onwind,True,10.954048,0.569543,0.015,109295.569252,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
1 onwind,1,onwind,True,2767.568457,218.413058,0.015,109295.569252,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
3 onwind,3,onwind,True,653.592418,72.784847,0.015,109295.569252,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
4 onwind,4,onwind,True,20317.910033,1974.184973,0.015,109295.569252,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
6 onwind,6,onwind,True,1913.852377,101.438862,0.015,109295.569252,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252 solar,252,solar,True,2400.214828,423.079545,0.010,55064.071419,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
253 solar,253,solar,True,110.273301,15.723475,0.010,55064.071419,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
254 solar,254,solar,True,380.635788,52.001619,0.010,55064.071419,1.0,PQ,,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [13]:
#I create the load file
import numpy as np
p_set=np.random.rand(len(n.snapshots),70)
p_set=p_set*100
df=pd.DataFrame(p_set)
load = df.set_index([n.snapshots]).rename_axis("time")
old_columns = list(df)

a=["bus 0", "bus 1", "bus 2", "bus 3", "bus 4", "bus 5", "bus 6", "bus 7", "bus 8", "bus 9", 
   "bus 10", "bus 11", "bus 12", "bus 13", "bus 14", "bus 15", "bus 16", "bus 17", "bus 18", "bus 19", 
   "bus 20", "bus 21", "bus 22", "bus 23", "bus 24", "bus 25", "bus 26", "bus 27", "bus 28", "bus 29", 
   "bus 30", "bus 31", "bus 32", "bus 33", "bus 34", "bus 35", "bus 36", "bus 37", "bus 38", "bus 39", 
   "bus 40", "bus 41", "bus 42", "bus 43", "bus 44", "bus 45", "bus 46", "bus 47", "bus 48", "bus 49", 
   "bus 50", "bus 51", "bus 52", "bus 53", "bus 54", "bus 55", "bus 56", "bus 57", "bus 58", "bus 59",
   "bus 60", "bus 61", "bus 62", "bus 63", "bus 64", "bus 65", "bus 66", "bus 67", "bus 68", "bus 69"]
   
new_col_list=a

# rename the columns:
load=load.rename(columns={old_columns[idx]: name for  (idx, name) in enumerate(new_col_list)})


In [14]:
def attach_load(n, load_paths):
    
    for i in range(buses):
   
        n.madd("Load", [i], bus=[i])
      


In [15]:
attach_load(n, load)

Index(['0'], dtype='object')
Index(['1'], dtype='object')
Index(['2'], dtype='object')
Index(['3'], dtype='object')
Index(['4'], dtype='object')
Index(['5'], dtype='object')
Index(['6'], dtype='object')
Index(['7'], dtype='object')
Index(['8'], dtype='object')
Index(['9'], dtype='object')
Index(['10'], dtype='object')
Index(['11'], dtype='object')
Index(['12'], dtype='object')
Index(['13'], dtype='object')
Index(['14'], dtype='object')
Index(['15'], dtype='object')
Index(['16'], dtype='object')
Index(['17'], dtype='object')
Index(['18'], dtype='object')
Index(['19'], dtype='object')
Index(['20'], dtype='object')
Index(['21'], dtype='object')
Index(['22'], dtype='object')
Index(['23'], dtype='object')
Index(['24'], dtype='object')
Index(['25'], dtype='object')
Index(['26'], dtype='object')
Index(['27'], dtype='object')
Index(['28'], dtype='object')
Index(['29'], dtype='object')
Index(['30'], dtype='object')
Index(['31'], dtype='object')
Index(['32'], dtype='object')
Index(['33'], dtype=

In [16]:
print(n)

PyPSA Network
Components:
 - Bus: 200
 - Generator: 449
 - Load: 200
Snapshots: 8760
